In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pickle

import numpy as np
import pandas as pd

import settings as conf

In [3]:
# genes_associations_dir = os.path.join(conf.PREPROCESSED_BASED_DIR, 'gene_associations')
# smultixcan_gene_association_dirs = os.path.join(genes_associations_dir, 'mashr')

output_dir = os.path.join(conf.DELIVERABLES_DIR, 'roc_validation', 'classifier_tables')
os.makedirs(output_dir, exist_ok=True)

# Load gene mappings

In [4]:
with open(os.path.join(conf.GENES_METADATA_DIR, 'genes_mapping_simplified-0.pkl'), 'rb') as f:
    genes_mapping_0 = pickle.load(f)

with open(os.path.join(conf.GENES_METADATA_DIR, 'genes_mapping_simplified-1.pkl'), 'rb') as f:
    genes_mapping_1 = pickle.load(f)

# Load S-MultiXcan results

In [5]:
spredixcan_genes_associations_filename = os.path.join(conf.GENE_ASSOC_DIR, 'smultixcan-mashr-zscores.pkl.xz')
display(spredixcan_genes_associations_filename)

spredixcan_genes_associations = pd.read_pickle(spredixcan_genes_associations_filename)

'/mnt/phenomexcan_base/gene_assoc/smultixcan-mashr-zscores.pkl.xz'

In [6]:
spredixcan_genes_associations.shape

(22515, 4091)

In [7]:
spredixcan_genes_associations.head(5)

,20096_1-Size_of_red_wine_glass_drunk_small_125ml,2345-Ever_had_bowel_cancer_screening,N49-Diagnoses_main_ICD10_N49_Inflammatory_disorders_of_male_genital_organs_not_elsewhere_classified,100011_raw-Iron,5221-Index_of_best_refractometry_result_right,20003_1141150624-Treatmentmedication_code_zomig_25mg_tablet,S69-Diagnoses_main_ICD10_S69_Other_and_unspecified_injuries_of_wrist_and_hand,20024_1136-Job_code_deduced_Information_and_communication_technology_managers,20002_1385-Noncancer_illness_code_selfreported_allergy_or_anaphylactic_reaction_to_food,G6_SLEEPAPNO-Sleep_apnoea,...,Astle_et_al_2016_Sum_basophil_neutrophil_counts,RA_OKADA_TRANS_ETHNIC,pgc.scz2,PGC_ADHD_EUR_2017,MAGIC_FastingGlucose,Astle_et_al_2016_Red_blood_cell_count,SSGAC_Depressive_Symptoms,BCAC_ER_positive_BreastCancer_EUR,IBD.EUR.Inflammatory_Bowel_Disease,Astle_et_al_2016_High_light_scatter_reticulocyte_count
gene_name,,,,,,,,,,,,,,,,,,,,,
ENSG00000000419,0.169468,0.102558,0.239545,0.887758,1.313448,1.472148,0.726160,1.516367,1.299771,1.068093,...,0.813014,0.275993,0.510834,0.024717,0.430951,0.824314,0.367414,1.377624,0.738444,0.298259
ENSG00000000457,1.358856,1.846875,0.139324,0.129530,0.757757,1.103979,0.612418,1.822327,2.035372,1.008058,...,1.441795,0.654791,2.545653,1.202984,0.514244,0.237223,0.414171,0.101731,1.012735,0.945167
ENSG00000000460,0.151008,1.173202,1.179426,0.571656,0.098771,0.221072,0.276415,0.461381,0.855502,0.201876,...,0.668962,0.300040,0.541782,1.033308,0.482261,0.695624,0.336480,0.083316,3.493196,0.991948
ENSG00000000938,1.302722,0.841524,1.578926,0.721340,0.139314,4.387016,0.125959,1.247123,0.215124,0.892083,...,0.126657,0.048048,1.886356,0.540496,0.127524,1.494501,0.056432,1.704863,1.351619,1.027297
ENSG00000000971,1.338813,0.262339,0.689379,1.702019,0.325859,0.063161,1.141126,0.882682,0.035533,1.810191,...,0.858497,1.675562,2.319072,1.598721,0.162958,0.005703,3.004544,0.803669,0.444266,0.165671


# Load OMIM silver standard

**FIXME**: update this silver standard including the new 42 traits?

In [8]:
omim_silver_standard = pd.read_csv(os.path.join(conf.DATA_DIR, 'omim_silver_standard.tsv'), sep='\t')

In [9]:
omim_silver_standard = omim_silver_standard.dropna(subset=['ensembl_gene_id', 'trait', 'pheno_mim'])

In [10]:
display(omim_silver_standard.shape)
display(omim_silver_standard.head())

(7809, 7)

,trait,pheno_mim,mim,entry_type,entrez_gene_id,approved_gene_symbol,ensembl_gene_id
0,M41-Diagnoses_main_ICD10_M41_Scoliosis,101800,188830,gene,5573,PRKAR1A,ENSG00000108946
1,M41-Diagnoses_main_ICD10_M41_Scoliosis,102500,600275,gene,4853,NOTCH2,ENSG00000134250
2,M41-Diagnoses_main_ICD10_M41_Scoliosis,105830,601623,gene,7337,UBE3A,ENSG00000114062
3,M41-Diagnoses_main_ICD10_M41_Scoliosis,108120,190990,gene,7169,TPM2,ENSG00000198467
4,M41-Diagnoses_main_ICD10_M41_Scoliosis,108145,613629,gene,63895,PIEZO2,ENSG00000154864


### Save list of mapped UK Biobank traits

In [11]:
from entity import Trait

In [12]:
ukb_traits_mapped = pd.Series(omim_silver_standard['trait'].unique())
display(ukb_traits_mapped.head())

0               M41-Diagnoses_main_ICD10_M41_Scoliosis
1            H80-Diagnoses_main_ICD10_H80_Otosclerosis
2    20002_1226-Noncancer_illness_code_selfreported...
3    I25-Diagnoses_main_ICD10_I25_Chronic_ischaemic...
4    20002_1136-Noncancer_illness_code_selfreported...
dtype: object

In [13]:
ukb_traits_mapped.shape

(107,)

In [14]:
ukb_codes_mapped = [Trait(full_code=t).code for t in ukb_traits_mapped]

In [15]:
len(ukb_codes_mapped)

107

In [16]:
ukb_codes_mapped[:5]

['M41', 'H80', '20002_1226', 'I25', '20002_1136']

In [17]:
# This is needed to run gwas2gene (Yanyu's scripts)
pd.Series(ukb_codes_mapped).to_csv('/mnt/tmp/selected_ukb_traits_omim.txt', index=False, header=False)

# Read gwas2gene results

These results were generated with scripts in `scripts/extras/gwas2gene`.

In [24]:
from glob import glob

import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()

In [26]:
readRDS = robjects.r['readRDS']

In [27]:
f_files = glob(os.path.join(conf.OMIM_SILVER_STANDARD_GWAS_TO_GENE_DIR, '*.rds'))
display(len(f_files))

if len(f_files) != len(ukb_codes_mapped):
    print(f'WARNING: some files are not there. {len(ukb_codes_mapped)} expected, {len(f_files)} found.')

99

In [28]:
gwas2genes_results = {}

for f in f_files:
    f_base = os.path.basename(f)
    f_code = f_base.split('.')[0]
    
    #print(f_base)
    rds_contents = readRDS(f)
    
    if len(rds_contents[1]) > 0:
        f_gene_list = list(rds_contents[1][0].iter_labels())
    else:
        print(f'{f_code}: empty')
        f_gene_list = []
    
    gwas2genes_results[f_code] = spredixcan_genes_associations.index.intersection(set(f_gene_list))

In [29]:
gwas2gene_all_genes = []

for k in gwas2genes_results.keys():
    gwas2gene_all_genes.extend(gwas2genes_results[k])

display(len(gwas2gene_all_genes))

gwas2gene_all_genes = set(gwas2gene_all_genes)
display(len(gwas2gene_all_genes))

# gwas2gene_all_genes = spredixcan_genes_associations.index.intersection(gwas2gene_all_genes)
# display(len(gwas2gene_all_genes))

19182

9505

# Create list of UKB-OMIM traits

In [30]:
omim_silver_standard.head()

,trait,pheno_mim,mim,entry_type,entrez_gene_id,approved_gene_symbol,ensembl_gene_id
0,M41-Diagnoses_main_ICD10_M41_Scoliosis,101800,188830,gene,5573,PRKAR1A,ENSG00000108946
1,M41-Diagnoses_main_ICD10_M41_Scoliosis,102500,600275,gene,4853,NOTCH2,ENSG00000134250
2,M41-Diagnoses_main_ICD10_M41_Scoliosis,105830,601623,gene,7337,UBE3A,ENSG00000114062
3,M41-Diagnoses_main_ICD10_M41_Scoliosis,108120,190990,gene,7169,TPM2,ENSG00000198467
4,M41-Diagnoses_main_ICD10_M41_Scoliosis,108145,613629,gene,63895,PIEZO2,ENSG00000154864


# Create PrediXcan classifier table

In [31]:
_tmp = omim_silver_standard[['trait', 'ensembl_gene_id']]
ukb_traits_common = _tmp['trait'].unique()

omim_true_classes = _tmp[['trait', 'ensembl_gene_id']].drop_duplicates()
omim_true_classes = omim_true_classes.assign(omim_value=1)
omim_true_classes = omim_true_classes.set_index(['trait', 'ensembl_gene_id'])

In [32]:
len(ukb_traits_common)

107

In [33]:
omim_true_classes.shape

(7046, 1)

In [34]:
omim_true_classes.head()

omim_value
trait                                  ensembl_gene_id            
M41-Diagnoses_main_ICD10_M41_Scoliosis ENSG00000108946           1
                                       ENSG00000134250           1
                                       ENSG00000114062           1
                                       ENSG00000198467           1
                                       ENSG00000154864           1

In [35]:
len(ukb_traits_common)

107

In [37]:
index_tuples = []

for t in ukb_traits_common:
    t_code = Trait(full_code=t).code
    if t_code not in gwas2genes_results:
        continue
    
    for g in gwas2genes_results[t_code]:
        index_tuples.append((t, g))

In [38]:
len(index_tuples)

19182

In [39]:
index_tuples[:5]

[('M41-Diagnoses_main_ICD10_M41_Scoliosis', 'ENSG00000012504'),
 ('M41-Diagnoses_main_ICD10_M41_Scoliosis', 'ENSG00000075089'),
 ('M41-Diagnoses_main_ICD10_M41_Scoliosis', 'ENSG00000075336'),
 ('M41-Diagnoses_main_ICD10_M41_Scoliosis', 'ENSG00000078401'),
 ('M41-Diagnoses_main_ICD10_M41_Scoliosis', 'ENSG00000090263')]

In [40]:
classifier_index = pd.MultiIndex.from_tuples(
    index_tuples,
    names=['ukb_efo', 'gene']
)

In [41]:
len(gwas2gene_all_genes)

9505

In [42]:
classifier_index.shape

(19182,)

In [43]:
predixcan_classifier_df = pd.DataFrame(index=classifier_index, columns=['score', 'predicted_class', 'true_class'])

In [44]:
predixcan_classifier_df = predixcan_classifier_df.sort_index()

In [45]:
predixcan_classifier_df.shape

(19182, 3)

In [46]:
predixcan_classifier_df['true_class'] = 0

In [47]:
predixcan_classifier_df.head()

score predicted_class  true_class
ukb_efo                     gene                                             
1200-Sleeplessness_insomnia ENSG00000001617   NaN             NaN           0
                            ENSG00000003756   NaN             NaN           0
                            ENSG00000004534   NaN             NaN           0
                            ENSG00000004838   NaN             NaN           0
                            ENSG00000004897   NaN             NaN           0

In [48]:
true_classes = omim_true_classes.squeeze()
display(true_classes.shape)
display(true_classes.head())

(7046,)

trait                                   ensembl_gene_id
M41-Diagnoses_main_ICD10_M41_Scoliosis  ENSG00000108946    1
                                        ENSG00000134250    1
                                        ENSG00000114062    1
                                        ENSG00000198467    1
                                        ENSG00000154864    1
Name: omim_value, dtype: int64

In [49]:
predixcan_classifier_df.loc[predixcan_classifier_df.index.intersection(true_classes.index), 'true_class'] = 1

In [50]:
assert predixcan_classifier_df['true_class'].isna().sum() == 0

In [51]:
# some testing
predixcan_classifier_df.loc[('M41-Diagnoses_main_ICD10_M41_Scoliosis',)].head()

,score,predicted_class,true_class
gene,,,
ENSG00000012504,NaN,NaN,0
ENSG00000075089,NaN,NaN,0
ENSG00000075336,NaN,NaN,0
ENSG00000078401,NaN,NaN,0
ENSG00000090263,NaN,NaN,0


In [52]:
true_classes.loc[('M41-Diagnoses_main_ICD10_M41_Scoliosis', 'ENSG00000112234')]

1

In [53]:
'ENSG00000090263' not in true_classes.loc['M41-Diagnoses_main_ICD10_M41_Scoliosis'].index

True

In [54]:
assert predixcan_classifier_df.loc[('M41-Diagnoses_main_ICD10_M41_Scoliosis', 'ENSG00000112234'), 'true_class'] == 1.0
assert predixcan_classifier_df.loc[('M41-Diagnoses_main_ICD10_M41_Scoliosis', 'ENSG00000090263'), 'true_class'] == 0.0

In [55]:
len(gwas2gene_all_genes)

9505

In [57]:
# score
df_score = pd.Series(index=classifier_index)

for trait in ukb_traits_common:
    trait_code = Trait(full_code=trait).code
    if trait_code not in gwas2genes_results:
        print(trait_code)
        continue
    trait_genes = gwas2genes_results[trait_code]
    scores = spredixcan_genes_associations.loc[trait_genes, trait]
    df_score.loc[trait] = scores.values

M13
I95
I71
20002_1264
20002_1081
22130
C80
20002_1538


In [58]:
# some testing
df_score = df_score.dropna().sort_index()
assert df_score.isna().sum().sum() == 0

In [59]:
df_score.head()

ukb_efo                      gene           
1200-Sleeplessness_insomnia  ENSG00000001617    5.986574
                             ENSG00000003756    5.540858
                             ENSG00000004534    5.270682
                             ENSG00000004838    1.168578
                             ENSG00000004897    0.042414
dtype: float64

In [60]:
# some testing
_gene, _trait = ('ENSG00000090263', 'M41-Diagnoses_main_ICD10_M41_Scoliosis')
assert spredixcan_genes_associations.loc[_gene, _trait] == df_score.loc[_trait, _gene]

In [61]:
_gene, _trait = ('ENSG00000070061', 'O14-Diagnoses_main_ICD10_O14_Gestational_pregnancyinduced_hypertension_with_significant_proteinuria')
assert spredixcan_genes_associations.loc[_gene, _trait] == df_score.loc[_trait, _gene]

In [62]:
df_score.shape

(19119,)

In [63]:
df_score.head()

ukb_efo                      gene           
1200-Sleeplessness_insomnia  ENSG00000001617    5.986574
                             ENSG00000003756    5.540858
                             ENSG00000004534    5.270682
                             ENSG00000004838    1.168578
                             ENSG00000004897    0.042414
dtype: float64

In [64]:
df_score.min()

9.034959836740552e-05

In [65]:
df_score.max()

40.0

In [66]:
predixcan_classifier_df = predixcan_classifier_df.assign(score=df_score)

In [68]:
# assert not predixcan_classifier_df['score'].isna().any()

In [69]:
from scipy import stats

In [70]:
_n_genes = len(gwas2gene_all_genes)
display(_n_genes)

_n_ukb_traits = len(ukb_traits_mapped)
display(_n_ukb_traits)

display(_n_genes * _n_ukb_traits)

PVALUE_THRESHOLD = (0.05 / (_n_genes * _n_ukb_traits))
display(PVALUE_THRESHOLD)

ZSCORE_THRESHOLD = np.abs(stats.norm.ppf(PVALUE_THRESHOLD / 2))
display(ZSCORE_THRESHOLD)

9505

107

1017035

4.916251653089618e-08

5.454313085046045

In [71]:
predixcan_classifier_df = predixcan_classifier_df.assign(predicted_class=(predixcan_classifier_df['score'] > ZSCORE_THRESHOLD).astype(int))

In [72]:
predixcan_classifier_df.shape

(19182, 3)

In [73]:
predixcan_classifier_df.head()

score  predicted_class  \
ukb_efo                     gene                                         
1200-Sleeplessness_insomnia ENSG00000001617  5.986574                1   
                            ENSG00000003756  5.540858                1   
                            ENSG00000004534  5.270682                0   
                            ENSG00000004838  1.168578                0   
                            ENSG00000004897  0.042414                0   

                                             true_class  
ukb_efo                     gene                         
1200-Sleeplessness_insomnia ENSG00000001617           0  
                            ENSG00000003756           0  
                            ENSG00000004534           0  
                            ENSG00000004838           0  
                            ENSG00000004897           0

In [74]:
predixcan_classifier_df.loc['M41-Diagnoses_main_ICD10_M41_Scoliosis'].sort_values('true_class', ascending=False).head()

,score,predicted_class,true_class
gene,,,
ENSG00000157764,0.258736,0,1
ENSG00000162769,0.086607,0,1
ENSG00000112234,0.270457,0,1
ENSG00000012504,0.643603,0,0
ENSG00000173208,0.162836,0,0


## Select genes per trait

In [75]:
#selected_predixcan_classifier_df = predixcan_classifier_df.loc[predixcan_classifier_df.index.intersection(trait_genes_to_keep)]
selected_predixcan_classifier_df = predixcan_classifier_df

In [76]:
# some testing

In [77]:
selected_predixcan_classifier_df.shape

(19182, 3)

In [78]:
selected_predixcan_classifier_df.head()

score  predicted_class  \
ukb_efo                     gene                                         
1200-Sleeplessness_insomnia ENSG00000001617  5.986574                1   
                            ENSG00000003756  5.540858                1   
                            ENSG00000004534  5.270682                0   
                            ENSG00000004838  1.168578                0   
                            ENSG00000004897  0.042414                0   

                                             true_class  
ukb_efo                     gene                         
1200-Sleeplessness_insomnia ENSG00000001617           0  
                            ENSG00000003756           0  
                            ENSG00000004534           0  
                            ENSG00000004838           0  
                            ENSG00000004897           0

In [80]:
selected_predixcan_classifier_df.sort_values('predicted_class', ascending=False).head()

score  \
ukb_efo                                            gene                         
1200-Sleeplessness_insomnia                        ENSG00000001617   5.986574   
20002_1453-Noncancer_illness_code_selfreported_... ENSG00000204618   7.450705   
                                                   ENSG00000204632  12.074117   
                                                   ENSG00000204644   8.480428   
                                                   ENSG00000204655   7.722879   

                                                                    predicted_class  \
ukb_efo                                            gene                               
1200-Sleeplessness_insomnia                        ENSG00000001617                1   
20002_1453-Noncancer_illness_code_selfreported_... ENSG00000204618                1   
                                                   ENSG00000204632                1   
                                                   ENSG00000204644                1   
                                                   ENSG00000204655                1   

                                                                    true_class  
ukb_efo                                            gene                         
1200-Sleeplessness_insomnia                        ENSG00000001617           0  
20002_1453-Noncancer_illness_code_selfreported_... ENSG00000204618           0  
                                                   ENSG00000204632           0  
                                                   ENSG00000204644           0  
                                                   ENSG00000204655           0

In [81]:
_tmp = selected_predixcan_classifier_df.sort_values(['true_class', 'ukb_efo'], ascending=False)
display(_tmp.shape)
display(_tmp[_tmp['true_class'] > 0].shape)
display(_tmp[_tmp['true_class'] > 0].head())

(19182, 3)

(125, 3)

score  \
ukb_efo                                            gene                        
R69-Diagnoses_main_ICD10_R69_Unknown_and_unspec... ENSG00000068305  0.540877   
                                                   ENSG00000140443  2.216935   
R40-Diagnoses_main_ICD10_R40_Somnolence_stupor_... ENSG00000133812  1.323981   
                                                   ENSG00000183287  1.210377   
N20-Diagnoses_main_ICD10_N20_Calculus_of_kidney... ENSG00000131183  6.246578   

                                                                    predicted_class  \
ukb_efo                                            gene                               
R69-Diagnoses_main_ICD10_R69_Unknown_and_unspec... ENSG00000068305                0   
                                                   ENSG00000140443                0   
R40-Diagnoses_main_ICD10_R40_Somnolence_stupor_... ENSG00000133812                0   
                                                   ENSG00000183287                0   
N20-Diagnoses_main_ICD10_N20_Calculus_of_kidney... ENSG00000131183                1   

                                                                    true_class  
ukb_efo                                            gene                         
R69-Diagnoses_main_ICD10_R69_Unknown_and_unspec... ENSG00000068305           1  
                                                   ENSG00000140443           1  
R40-Diagnoses_main_ICD10_R40_Somnolence_stupor_... ENSG00000133812           1  
                                                   ENSG00000183287           1  
N20-Diagnoses_main_ICD10_N20_Calculus_of_kidney... ENSG00000131183           1

### Test classes

In [82]:
selected_predixcan_classifier_df.index.get_level_values('ukb_efo').unique().shape

(99,)

In [83]:
selected_predixcan_classifier_df.index.get_level_values('gene').unique().shape

(9505,)

In [84]:
_pheno = 'N20-Diagnoses_main_ICD10_N20_Calculus_of_kidney_and_ureter'

In [85]:
_clinvar_asthma_genes = omim_silver_standard[omim_silver_standard['trait'] == _pheno]['ensembl_gene_id'].unique()
display(_clinvar_asthma_genes)
display(_clinvar_asthma_genes.shape)

array(['ENSG00000075891', 'ENSG00000130600', 'ENSG00000269821',
       'ENSG00000129757', 'ENSG00000143473', 'ENSG00000275410',
       'ENSG00000019186', 'ENSG00000134371', 'ENSG00000036828',
       'ENSG00000088256', 'ENSG00000187091', 'ENSG00000122194',
       'ENSG00000138592', 'ENSG00000040531', 'ENSG00000138079',
       'ENSG00000021488', 'ENSG00000090402', 'ENSG00000131482',
       'ENSG00000137700', 'ENSG00000164007', 'ENSG00000113946',
       'ENSG00000116039', 'ENSG00000168000', 'ENSG00000123191',
       'ENSG00000171365', 'ENSG00000165704', 'ENSG00000042753',
       'ENSG00000075643', 'ENSG00000169692', 'ENSG00000109667',
       'ENSG00000131183', 'ENSG00000109062', 'ENSG00000149257',
       'ENSG00000198931', 'ENSG00000157388', 'ENSG00000167207',
       'ENSG00000124827', 'ENSG00000134873'], dtype=object)

(38,)

In [86]:
_tmp = selected_predixcan_classifier_df.loc[_pheno]
_tmp.loc[_tmp.index.intersection(_clinvar_asthma_genes)]

,score,predicted_class,true_class
ENSG00000131183,6.246578,1,1


In [87]:
_predixcan_asthma_genes = selected_predixcan_classifier_df.loc[_pheno]

In [88]:
_predixcan_asthma_genes.head()

,score,predicted_class,true_class
gene,,,
ENSG00000005187,1.587926,0,0
ENSG00000005189,0.457434,0,0
ENSG00000011638,0.059849,0,0
ENSG00000027847,1.586332,0,0
ENSG00000048140,0.688334,0,0


In [89]:
selected_predixcan_classifier_df.shape

(19182, 3)

In [90]:
selected_predixcan_classifier_df['predicted_class'].value_counts()

0    17462
1     1720
Name: predicted_class, dtype: int64

In [91]:
selected_predixcan_classifier_df['true_class'].value_counts()

0    19057
1      125
Name: true_class, dtype: int64

In [92]:
selected_predixcan_classifier_df.sort_values(['true_class'], ascending=[False])

,,score,predicted_class,true_class
ukb_efo,gene,,,
20002_1471-Noncancer_illness_code_selfreported_atrial_fibrillation,ENSG00000145362,0.019544,0,1
20002_1065-Noncancer_illness_code_selfreported_hypertension,ENSG00000132170,1.066410,0,1
22127-Doctor_diagnosed_asthma,ENSG00000232810,1.996019,0,1
M05-Diagnoses_main_ICD10_M05_Seropositive_rheumatoid_arthritis,ENSG00000204498,3.601510,0,1
20002_1226-Noncancer_illness_code_selfreported_hypothyroidismmyxoedema,ENSG00000103449,0.199143,0,1
2453-Cancer_diagnosed_by_doctor,ENSG00000110092,1.888766,0,1
20002_1226-Noncancer_illness_code_selfreported_hypothyroidismmyxoedema,ENSG00000166037,1.162134,0,1
20002_1065-Noncancer_illness_code_selfreported_hypertension,ENSG00000008441,2.760524,0,1
I48-Diagnoses_main_ICD10_I48_Atrial_fibrillation_and_flutter,ENSG00000145362,0.554050,0,1


# Save classifier table

In [97]:
# remove nans
selected_predixcan_classifier_df = selected_predixcan_classifier_df.dropna()

In [98]:
selected_predixcan_classifier_df.head()

score  predicted_class  \
ukb_efo                     gene                                         
1200-Sleeplessness_insomnia ENSG00000001617  5.986574                1   
                            ENSG00000003756  5.540858                1   
                            ENSG00000004534  5.270682                0   
                            ENSG00000004838  1.168578                0   
                            ENSG00000004897  0.042414                0   

                                             true_class  
ukb_efo                     gene                         
1200-Sleeplessness_insomnia ENSG00000001617           0  
                            ENSG00000003756           0  
                            ENSG00000004534           0  
                            ENSG00000004838           0  
                            ENSG00000004897           0

In [99]:
selected_predixcan_classifier_df.shape

(19119, 3)

In [100]:
selected_predixcan_classifier_df.to_csv(
    os.path.join(output_dir, 'smultixcan-mashr-classifier_data.tsv.gz'),
    sep='\t', index=False
)